# User-defined LAMMPS potentials

This Notebook provides details and demonstrations about creating PotentialLAMMPS objects for potentials not currently in the database.  This allows for users to easily integrate their personal LAMMPS potentials into the potentials database or any other tools that use the potentials package.

_Version 0.4 update_: Starting with version 0.4, the process of building new PotentialLAMMPS objects has been greatly streamlined by merging the building tools into the class itself and making the class attributes modifiable as well.  This Notebook has been completely updated to reflect these changes. 

In [1]:
from pathlib import Path
import potentials
from potentials.record.PotentialLAMMPS import PotentialLAMMPS

print('Notebook tested for potentials version', potentials.__version__)

Notebook tested for potentials version 0.4.0


## 1. Defining a new LAMMPS potential

The PotentialLAMMPS objects and the associated potential_LAMMPS records are designed to collect metadata for a LAMMPS implementation of a potential.  This metadata broadly falls into three categories:

1. Information detailing how to dynamically generate the LAMMPS command lines for the potential.
2. Metadata for storing the implementation information in a potentials Database.
3. Additional metadata for further characterization and identification.

In general, the #1 category is all that is needed for a user-defined LAMMPS potential to be integrated into most tools and to run a LAMMPS simulation with the potential.  However, at least some of the #2 category must be defined if you wish to save the information to a local database allowing full integration of the user-defined LAMMPS potential with all of the potentials package's database tools.  Finally, the #3 category is icing on the cake which mostly corresponds to releasing the record publicly.

The primary challenge with the #1 category is that the LAMMPS pair_coeff lines do not have a common universal format across all of the pair_styles.  To help address this, there are PotentialLAMMPS class methods that help automatically build the correct pair_coeff content for many of the pair_styles. 

### 1.1. Common LAMMPS generation parameters

These are the parameters for the #1 category that are common to all pair_styles and initialization methods.

- __pair_style__ (*str*) The LAMMPS pair_style option.
- __units__ (*str*) The LAMMPS option option. Default value is 'metal'.
- __atom_style__ (*str*) The LAMMPS atom_style option. Default value is 'atomic'.
- __symbols__ (*str or list*) The atomic model symbols defined by the potential.  Optional if the symbol tags match elemental tags and the elemental tags are given.
- __elements__ (*str or list*) The elemental symbols associated with each symbol model, if any.
- __masses__ (*float or list*) The atomic masses to use for each symbol model. Optional if elements are given a the masses will default to standard atomic mass values.
- __charges__ (*float or list*) Constant atomic charges to assign to each symbol model.  Default values are zero. Note that this is only used for ionic potentials with constant charges and not for variable charge potentials. Default values are 0.0 for each symbol.  the object has been created.

### 1.2. paramfile()

The PotentialLAMMPS.paramfile() method is for the most common format for potentials that rely on parameter files.  It is used by nearly all potentials that read in a single parameter file and associate atom types to model symbols.  This will work for any potential that has a single pair_coeff line of the following format

    pair_coeff * * <paramfile> <list of model symbols>

Method-specific parameters

- __paramfile__ (*str*) The name of the potential parameter file.

In [2]:
# Define potential for Al.eam.alloy
potential = PotentialLAMMPS.paramfile(pair_style='eam/alloy', elements='Al', paramfile='Al.eam.alloy')

print(potential.pair_info())


pair_style eam/alloy 
pair_coeff * * Al.eam.alloy Al

mass 1 26.9815385




### 1.3. meam()

The PotentialLAMMPS.meam() method is for pair styles like meam that use two types of parameter files: a general library file and a model-specific parameter file.  This will work for any potential that has a single pair_coeff line of the following format

    pair_coeff * * <libfile> <list of libfile symbols> <paramfile> <list of model symbols>

Method-specific parameters

- __libfile__ (*str*) The name of the potential library file.
- __paramfile__ (*str, optional*) The name of the potential parameter file. Might not be used by older MEAM potentials.

*Note*: The MEAM parameter files for alloy (multi-element) models need to know which particle models to associate their parameters with.  As such, the first list of symbols in the pair_coeff line must be in a specific order.  The builder will use the order of symbols/elements parameters as given.

In [3]:
# Define MEAM potential using the lib.meam and Al.meam parameter file
potential = PotentialLAMMPS.meam(pair_style='meam', elements='Al', libfile='lib.meam', paramfile='Al.meam')

print(potential.pair_info())


pair_style meam 
pair_coeff * * lib.meam Al Al.meam Al

mass 1 26.9815385




In [4]:
# Define MEAM potential using the lib.meam only
potential = PotentialLAMMPS.meam(pair_style='meam', elements='Al', libfile='lib.meam')

print(potential.pair_info())


pair_style meam 
pair_coeff * * lib.meam Al NULL Al

mass 1 26.9815385




### 1.4. eam()

The PotentialLAMMPS.eam() method is for the original eam pair_style which uses a unique format due to it defining elemental models in parameter files and using a universal mixing function.  This constructs pair_coeff lines of the format

    pair_coeff 1 1 <paramfiles[0]>
    pair_coeff 2 2 <paramfiles[1]>
    ...

Method-specific parameters

- __paramfiles__ (*list*) The names of the elemental potential parameter files. There should be exactly one parameter file for each symbol model.

In [5]:
# Define potential using the Al.eam and Cu.eam parameter files
potential = PotentialLAMMPS.eam(pair_style='eam', elements=['Al', 'Cu'], paramfiles=['Al.eam', 'Cu.eam'])

print(potential.pair_info())


pair_style eam 
pair_coeff 1 1 Al.eam
pair_coeff 2 2 Cu.eam

mass 1 26.9815385
mass 2 63.546




### 1.5. eim()

The PotentialLAMMPS.eim() method is for the eim pair_style, which has the following pair_coeff format

    pair_coeff * * <list of paramfile symbols> <paramfile> <list of model symbols>

Method-specific parameters

- __paramfile__ (*str*) The name of the potential parameter file.

*Note*: The EIM parameter files for alloy (multi-element) models need to know which particle models to associate their parameters with.  As such, the first list of symbols in the pair_coeff line must be in a specific order.  The builder will use the order of symbols/elements parameters as given.

In [6]:
potential = PotentialLAMMPS.eim(pair_style='eim', elements=['Al', 'Cu'], paramfile='AlCu.eim')

print(potential.pair_info())


pair_style eim 
pair_coeff * * Al Cu AlCu.eim Al Cu

mass 1 26.9815385
mass 2 63.546




### 1.6. Pair potentials

LAMMPS pair_styles for pair potentials tend to be of the following format, where each pair_coeff line defines the potential parameters for interactions between two atom type species 

    pair_coeff 1 1 <a_11> <b_11> ...
    pair_coeff 1 2 <a_12> <b_12> ...
    ...

Defining a PotentialLAMMPS object for such potentials can be done in three steps:

1. Initialize the PotentialLAMMPS object and define the common parameters listed in 1.1.
2. Use add_pair_coeff() to create a new pair_coeff line for a given combination of symbol models.
3. Add all necessary terms to each pair_coeff line.

One thing to note with this is that each pair_coeff defines the interaction using symbol tags rather than integer atomic types.  This allows for the dynamic matching of atom types to the symbol tags when the LAMMPS command lines are built in a fashion similar to the parameter file-based variations above.

In [7]:
# Define pair potential with elements Al and Cu
potential = PotentialLAMMPS(pair_style='lj/cut', elements=['Al', 'Cu'])

# Specify coefficients for each unique interaction (Note: values are junk NOT REAL POTENTIALS!)
potential.add_pair_coeff(interaction=['Al', 'Al'])
potential.pair_coeffs[0].add_term('parameter', 12.14)
potential.pair_coeffs[0].add_term('parameter', 2.9)

potential.add_pair_coeff(interaction=['Cu', 'Al'])
potential.pair_coeffs[1].add_term('parameter', 51.7)
potential.pair_coeffs[1].add_term('parameter', 9.9)

potential.add_pair_coeff(interaction=['Cu', 'Cu'])
potential.pair_coeffs[2].add_term('parameter', 19.4)
potential.pair_coeffs[2].add_term('parameter', 4.2)

print(potential.pair_info())


pair_style lj/cut 
pair_coeff 1 1 12.14 2.9
pair_coeff 1 2 51.7 9.9
pair_coeff 2 2 19.4 4.2

mass 1 26.9815385
mass 2 63.546




### 1.7 Other potentials, extra options

While the above descriptions capture the vast majority of pair_styles, many potentials require more custom definitions.  This section specifies how to properly build the metadata for generating the LAMMPS command lines for any arbitrary potential.

#### 1.7.1. allsymbols option

The allsymbols option is a boolean value that can be set during initialization or anytime after and is required by some pair_styles.

- If allsymbols is False (default), then the generated LAMMPS command lines will only be for the symbol models that are actively tied to an atom type in the atomic system.
- If allsymbols is True, then the generated LAMMPS command lines will include all symbol models defined by the potential regardless of if they are actively tied to an atom type in the atomic system. The unused symbol models will be associated with atom types beyond what is specified as being part of the system.  __Caution__: when defining your system in LAMMPS, make sure that the total atom types matches the full number being defined, not just those being used. This is done either during the create_box command, or as value in a data configuration file being read in with read_data.  

In [8]:
# Demonstrate the default nature of a multi-element potential applied to a system with only one element
potential = PotentialLAMMPS.paramfile(pair_style='eam/alloy', elements=['Al', 'Cu', 'Ni'], paramfile='AlCuNi.eam.alloy')
print(potential.pair_info('Al'))


pair_style eam/alloy 
pair_coeff * * AlCuNi.eam.alloy Al

mass 1 26.9815385




In [9]:
# Demonstrate how setting allsymbols will include all elements even when not used
potential = PotentialLAMMPS.paramfile(pair_style='eam/alloy', elements=['Al', 'Cu', 'Ni'], paramfile='AlCuNi.eam.alloy', allsymbols=True)
print(potential.pair_info('Al'))


pair_style eam/alloy 
pair_coeff * * AlCuNi.eam.alloy Al Cu Ni

mass 1 26.9815385
mass 2 63.546
mass 3 58.6934




#### 1.7.2. Additional pair_style terms and command lines

The pair_style value should be simply the primary pair_style option for the potential making the option queryable. However, some pair_styles have extra settings that are specified in the LAMMPS pair_style command line after the primary option is specified.  Additionally, some potentials require extra LAMMPS commands to modify other settings or add additional fixes. Both of these are managed by CommandLine objects that collect the command line terms and classify them all according to a set of pre-defined types.

The supported command line term types are:
- __option__ is the most general term and has a string value.
- __parameter__ is for numerical terms allowing them to be differentiated from the more general options.
- __file__ is for terms that point to a file or a directory.  The value here should be just the file name. When the LAMMPS commands are generated, all file terms will be converted to full paths with the pot_dir value as the parent directory. 
- __symbols__ is a boolean flag setting that when True will generate a list of the symbol models that correspond to the integer atom types. This is used by pair_coeff lines (see below).
- __symbolsList__ is a boolean flag setting that when True will generate a list of all of the unique symbol models being used.  This type is likely obsolete and should be depreciated.

__PotentialLAMMPS.pair_style_terms__ is a CommandLine object that manages the extra terms that appear on the pair_style line.

In [10]:
potential = PotentialLAMMPS(pair_style='lj/cut', elements=['Au'])

# Add cutoff value to the pair_style line
potential.pair_style_terms.add_term('parameter', 2.5)

# Add a pair_coeff for all atom types
potential.add_pair_coeff()
potential.pair_coeffs[0].add_term('parameter', 1)
potential.pair_coeffs[0].add_term('parameter', 1)

print(potential.pair_info())


pair_style lj/cut 2.5
pair_coeff * * 1.0 1.0

mass 1 196.966569




__PotentialLAMMPS.Commands__ is a list of CommandLine objects, with each object representing an additional command line that should be used.  

In [11]:
# add_command() creates a new command line for the above potential
potential.add_command()

# Once created, terms can be added.  Here, a single "option" is used to represent the full command, but it could be separated into multiple terms as shown above. 
potential.commands[0].add_term('option', 'pair_modify shift yes')

print(potential.pair_info())


pair_style lj/cut 2.5
pair_coeff * * 1.0 1.0

mass 1 196.966569

pair_modify shift yes



#### 1.7.3. pair_coeff lines

The pair_coeff lines are managed by objects of the PairCoeffLine class, which is a subclass of CommandLine. In particular, PairCoeffLine allows for the defining of the command line terms and links them to an __interaction__ list of symbol models. This process largely follows what is shown above in sections 1.6. and 1.7.2.

The rules for the number of symbols to include in the interaction list depend on the type of potential and the interaction it represents.

- If no interaction symbols are given, the pair_coeff line is assumed to be applied to all symbols, i.e. the line will use the * * wildcards.  
- For pair potentials, each interaction should have exactly two symbols indicating the two atom models involved.
- For the classic eam pair_style, each interaction should be two identical symbols.
- For hybrid styles, the interaction list should be the symbols associated with that component of the potential. The missing types will be automatically filled in with NULL values as needed.

In [12]:
# Hybrid potential example #1

# Define with main pair_style option
potential = PotentialLAMMPS(pair_style='hybrid/overlay', elements='Ta')

# Add extra pair_style terms - separation into terms is arbitrary but done here for clarity
potential.pair_style_terms.add_term('option', 'zbl 4 4.8')
potential.pair_style_terms.add_term('option', 'snap')

# Define the zbl pair_coeff line
potential.add_pair_coeff()
potential.pair_coeffs[0].add_term('option', 'zbl')
potential.pair_coeffs[0].add_term('parameter', 73)
potential.pair_coeffs[0].add_term('parameter', 73)

# Define the snap pair_coeff line
potential.add_pair_coeff()
potential.pair_coeffs[1].add_term('option', 'snap')
potential.pair_coeffs[1].add_term('file', 'Ta06A.snapcoeff')
potential.pair_coeffs[1].add_term('file', 'Ta06A.snapcparam')
potential.pair_coeffs[1].add_term('symbols', True)


print(potential.pair_info())


pair_style hybrid/overlay zbl 4 4.8 snap
pair_coeff * * zbl 73.0 73.0
pair_coeff * * snap Ta06A.snapcoeff Ta06A.snapcparam Ta

mass 1 180.94788




In [13]:
# Hybrid potential example #2

# Define with main pair_style option
potential = PotentialLAMMPS(pair_style='hybrid', elements=['Si', 'C'])

# Add extra pair_style terms - separation into terms is arbitrary but done here for clarity
potential.pair_style_terms.add_term('option', 'lj/cut 2.5')
potential.pair_style_terms.add_term('option', 'tersoff')
potential.pair_style_terms.add_term('option', 'tersoff')

# Define the lj/cut pair_coeff line
potential.add_pair_coeff(interaction=['Si', 'C'])
potential.pair_coeffs[0].add_term('option', 'lj/cut')
potential.pair_coeffs[0].add_term('parameter', 1.0)
potential.pair_coeffs[0].add_term('parameter', 1.5)

# Define the first tersoff pair_coeff line
potential.add_pair_coeff(interaction='Si')
potential.pair_coeffs[1].add_term('option', 'tersoff')
potential.pair_coeffs[1].add_term('file', 'Si.tersoff')
potential.pair_coeffs[1].add_term('symbols', True)

# Define the second tersoff pair_coeff line
potential.add_pair_coeff(interaction='C')
potential.pair_coeffs[2].add_term('option', 'tersoff')
potential.pair_coeffs[2].add_term('file', 'C.tersoff')
potential.pair_coeffs[2].add_term('symbols', True)

print(potential.pair_info())


pair_style hybrid lj/cut 2.5 tersoff tersoff
pair_coeff 1 2 lj/cut 1.0 1.5
pair_coeff * * tersoff Si.tersoff Si NULL
pair_coeff * * tersoff C.tersoff NULL C

mass 1 28.085
mass 2 12.0106




### 1.8. Saving the potential parameter information

After setting the content to define the potential information, you can call build_model() to generate the data model representation of the content and then save it as either JSON or XML. This content can then be read back in to instantly define the potential.

In [14]:
# build_model builds the model and returns the contents. The built contents can also be accessed later using the PotentialLAMMPS.model attribute
model = potential.build_model()
print(model == potential.model)

True


In [15]:
# Save to a json file
with open('mypot.json', 'w') as f:
    model.json(fp=f, indent=4)

In [16]:
# Initialize a new potential object from the JSON contents
potential2 = PotentialLAMMPS(model='mypot.json')

print(potential2.pair_info())


pair_style hybrid lj/cut 2.5 tersoff tersoff
pair_coeff 1 2 lj/cut 1.0 1.5
pair_coeff * * tersoff Si.tersoff Si NULL
pair_coeff * * tersoff C.tersoff NULL C

mass 1 28.085
mass 2 12.0106




## 2. Database metadata

The above allows you to define a LAMMPS potential implementation and to save the content for later. With a few more parameters, you can set it up so that your user-defined LAMMPS potentials can be integrated into the local potentials database and then be loaded and queried alongside the potentials from NIST.

Additional parameters supporting database 

- __id__ (*str, optional*) A human-readable identifier to name the LAMMPS potential implementation.
- __key__ (*str, optional*) A UUID4 code to uniquely identify the LAMMPS potential implementation.
- __name__ (*str, optional*) The name to save/refer to the LAMMPS potential implementation as when saved in a database. During initialization, this will be set to id if id is given and name is not.
- __potid__ (*str, optional*) A human-readable identifier to refer to the conceptual potential model that the potential is based on.  This should be shared by alternate implementations of the same potential.
- __potkey__ (*str, optional*) A UUID4 code to uniquely identify the conceptual potential model. This should be shared by alternate implementations of the same potential.

__Notes__: The name value must be set before the record can be saved to a database. It is highly recommended that id, key, and name all be set and that id == name as some tools may assume these things.

In [19]:
potential = PotentialLAMMPS.paramfile(pair_style='eam/alloy', paramfile='Al.eam.alloy', elements='Al', id='My--Al--potential--test')

In [20]:
potential.name

'My--Al--potential--test'

Note that the two id fields have null values.  They can be set either during initialization or as attributes after initialization.

### 2.2 Save locally

Any user-defined records for LAMMPS potentials can be saved locally and then later retrieved using potentials.Database.

In [21]:
# Create a dummy parameter file "Al.eam.alloy" 
dummyfile = Path('Al.eam.alloy')

with open(dummyfile, 'w') as f:
    f.write('Dummy parameter file Al.eam.alloy\n')
    f.write('This is created here to demonstrate how to associate parameter files with saved LAMMPS potentials\n')
    f.write('Give paths to already existing parameter files for real potentials\n')
    f.write('This is where parameters would appear if this was a real potential...')

#### 2.2.1 Database.save_lammps_potentials()

Saves a new LAMMPS potential to the local copy of the database

- __lammps_potentials__ (*PotentialLAMMPS or list of PotentialLAMMPS*) The lammps_potential(s) to save.
- __filenames__ (*list, optional*) The path names to any parameter files associated with each lammps_potentials.  Length of the list should be the same as the length of lammps_potentials.  Each entry can be None, a path, or a list of paths.  If the value is None for an entry then the corresponding Potential record will be searched for parameter files to download.   Note that files will only be copied/downloaded if the associated record is new/different.
- __format__ (*str, optional*) The file format to save the record files as.  Allowed values are 'xml' (default) and 'json'.
- __localpath__ (*path-like object, optional*) Path to a local directory where the record and files will be saved to. If not given, will use the localpath value set during object initialization.
- __indent__ (*int, optional*) The indentation spacing size to use for the locally saved record files. If not given, the JSON/XML content will be compact.
- __overwrite__ (*bool, optional*) If True (default) then any matching LAMMPS potentials already in the localpath will be updated if the content has changed.  If False, all existing LAMMPS potentials in the localpath will be unchanged.
- __verbose__ (*bool, optional*) If True, info messages will be printed during operations.  Default value is False.
- __reload__ (*bool, optional*) If True, will call load_lammps_potentials() after adding the new potential.  Default value is False: loading is slow and should only be done when you wish to retrieve the saved potentials.


In [23]:
# Initialize a Database with localpath testlibrary
potdb = potentials.Database()

# Save to local
potdb.save_lammps_potential(potential, filenames='Al.eam.alloy', verbose=True)

potential_LAMMPS record named My--Al--potential--test added to /home/lmh1/library
files saved to local folder


#### 2.2.2 load and get

Any local potentials in the Database's localpath can then be found after loading the lammps potentials into memory.

In [25]:
# get_lammps_potential will search through the loaded potentials
pot = potdb.get_lammps_potential(id='My--Al--potential--test')
print(pot)

potential_LAMMPS record named My--Al--potential--test


In [26]:
print(pot.pair_info())


pair_style eam/alloy 
pair_coeff * * My--Al--potential--test/Al.eam.alloy Al

mass 1 26.9815385




#### 2.2.3. Delete records

Any LAMMPS potentials saved can also be deleted with the delete_lammps_potential() method.

Parameters
    
- __lammps_potential__ (*PotentialLAMMPS or str*) The LAMMPS potential to delete from the remote database.  Can either give the corresponding PotentialLAMMPS object or just the potential's id/title.
- __local__ (*bool, optional*) If True (default) then the record will be deleted from the localpath.
- __remote__ (*bool, optional*) If True then the record will be deleted from the remote database.   Requires write permissions to potentials.nist.gov.  Default value is False.
- __localpath__ (*path-like object, optional*) Path to a local directory where the file to delete is located.  If not given, will use the localpath value set during object initialization.
- __verbose__ (*bool, optional*) If True, info messages will be printed during operations.  Default value is False.

In [27]:
potdb.delete_lammps_potential(pot, local=True, remote=False, verbose=True)

potential_LAMMPS record named My--Al--potential--test deleted from /home/lmh1/library


## 3. Additional metadata

These are the additional metadata content that can be set but which are unlikely to be useful for most user-defined potentials. This content is mostly in support of potential_LAMMPS records and contents stored in public databases like potentials.nist.gov.

- __url__ (*str, optional*) A web URL associated with a public database where the potential_LAMMPS record can be found.
- __poturl__ (*str, optional*) A web URL associated with a public database where the Potential record that the LAMMPS implementation is associated with can be found.
- __status__ (*str, optional*) Indicates the status of this particular implementation/version of the potential. Default None indicates the implementation to be active and current, 'superseded' indicates that it is a valid model, but a better implementation exists that should be used instead, and 'retracted' indicates that the implementation is a flawed invalid implementation that does not correctly represent the parent potential concept.
- __comments__ (*str, optional*) Descriptive information about the potential.  This will be included as a LAMMPS print command when pair_info is generated.
- __dois__ (*list, optional*) Any DOIs associated with the potential. These will be included in LAMMPS print commands when pair_info is generated.
- __artifacts__ (*lost, optional*) Artifact objects detailing any associated parameter or data files and the URLs where they can be downloaded from.  Including this makes it possible for the PotentialLAMMPS objects to download copies of the parameter files. These can be added one at a time with add_artifact().

In [28]:
# Delete the generated local example files
Path('mypot.json').unlink()
dummyfile.unlink()